In [ ]:
# 如果本地没有安装工具包，请去掉“#”，再执行
# !pip install -i https://mirrors.aliyun.com/pypi/simple/ xlwt

**<font color=blue>读取Excel的内容</font>**<br/>
<img src="images/Excel工资单明细.png" align="left"></img>

**<font color=blue>新建Excel的内容（合并单元格）</font>**<br/>
**<font color=red>为每个员工创建一个工作表，工作表名是员工姓名</font>**<br/>
<img src="images/Excel工资单_02.png" align="left"></img>


# 读取Excel，获取字段名

In [ ]:
# 导入Excel读写工具包
import xlrd
import xlwt

# 从（from）本地 xlwt_styles.py（.py扩展名省略）文件中，导入（import）创建样式的函数（create_style）
from xlwt_styles import create_style

# 定义两个变量（xls_filename_rd、xls_filename_wt）
xls_filename_rd = "./files/Excel_工资单明细.xls" # 读取的 Excel 文件名
xls_filename_wt = "./files/Excel_员工工资单_02.xls" # 新建的 Excel 文件名
# *******************************
# 虽然变量命名只要符合命名规则即可，但建议变量命名最好要有意义，对读懂代码很有帮助
# xls:Excel扩展名  filename:文件名  rd:read读取  wt:write写入
# *******************************

# 读取 Excel（xls_filename_rd 变量刚才已经定义），得到工作簿，赋值给变量 workbook_rd
workbook_rd = xlrd.open_workbook(xls_filename_rd)

# 获取第一个工作表，赋值给变量 sheet_rd
sheet_rd = workbook_rd.sheet_by_index(0)

# 获取工作表中表格的所有字段（第一行是字段名），记住后面会用到
field_names = sheet_rd.row_values(0)
print("字段名：",field_names)  # 打印一下看看字段名

# 读取Excel，获取员工姓名和明细数据

In [ ]:
# 创建两个“空”列表变量
staff_names = [] # 员工姓名列表
staff_values_list = [] # 员工数据明细列表（这个就是列表中的列表）

# 因为第一行（索引=0）是字段名，所以从第二行（索引=1）开始读数据
for row_index in range(1, sheet_rd.nrows):
    values = sheet_rd.row_values(row_index)  # 获取员工的数据列表
    # print(values)           # 可以删除注释，打印这一行数据列表看看
    staff_name = values[2]    # 第三列是姓名
    # print(staff_name)       # 可以删除注释，打印一下看看
    staff_names.append(staff_name)  # append
    staff_values_list.append(values) # 列表中可以再包含列表 [[...],[...]]

# 打印读取的员工姓名列表、员工数据明细列表
# print("员工姓名列表：",staff_names)           # 可以删除注释，打印一下看看
# print("员工数据明细列表：",staff_values_list) # 可以删除注释，打印一下看看

# 新建 Excel，创建样式

In [ ]:
# 创建写入工作簿对象
workbook_wt = xlwt.Workbook(encoding='utf-8')

# 看看要生成的表格样式，共有4个
# 合并单元格的样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=居中）
merge_style = create_style(back_color=57, font_color=1, font_size=12, horz=2)  # 标题显示用黄色背景图案
# 字段样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=靠右）
field_style = create_style(back_color=57, font_color=1, font_size=12, horz=3)
# 数据值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=居中）
value_style = create_style(back_color=1, font_color=0, font_size=12, horz=2)
# 金额值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=靠右，人民币格式）
rmb_style = create_style(back_color=1, font_color=0, font_size=12, horz=3, num_format_str="￥#,##0.00;￥-#,##0.00")

**<font color=blue>Excel颜色索引对照表</font>**<br/>
<img src="images/Excel颜色对照表.png" align="left"></img>

# 语法：条件判断

如果条件成立，则我要运行代码A，如果条件不成立，我就运行代码B
```python
if 条件?:
   run(A)
else:
   run(B)
```

In [ ]:
import random
# 每次生成的结果不一样（1 到 10）
num = random.randint(1,10)
print("num 等于", num)
if num >= 5:
    print("num 大于等于 5")
else:
    print("num 小于 5")

# 两个循环写入数据 ，保存 Excel

In [ ]:
# 第一循环：遍历每个员工的姓名，每个姓名创建一个工作表
for staff_index, staff_name in enumerate(staff_names):
    # 创建工作表，工作表名就是员工姓名
    sheet_wt_staff = workbook_wt.add_sheet(staff_name)

    # 获取员工的数据明细
    staff_data = staff_values_list[staff_index]
    # 第二循环：遍历字段和数据
    for field_index, field_name in enumerate(field_names):
        row = field_index + 1 # field_index 从 0 开始，则 row 从 1 开始

        if field_index < 14: # “实发”是最后一个字段名（索引=14），忽略不写入，后面合并单元格再写入
            # 第3列（c=2）写入字段名
            sheet_wt_staff.write(r=row, c=2, label=field_name, style=field_style)
        else:
            pass # 通过，这个代码什么都不干，对于“实发”字段名就不写入了

        value = staff_data[field_index]

        if field_index <= 2:   # 0，1，2 三个字段分别是月份、部门、姓名
            sheet_wt_staff.write(r=row, c=3, label=value, style=value_style) # 第4列（c=3）写入数据
        else:    # 3、4、5...都是金额字段，用RMB货币样式            
            sheet_wt_staff.write(r=row, c=3, label=value, style=rmb_style) # 第4列（c=3）写入数据

    # 设置字段列和数据列的列宽 256 × 宽度默认单位（字符）
    sheet_wt_staff.col(1).width = 256 * 12
    sheet_wt_staff.col(2).width = 256 * 15
    sheet_wt_staff.col(3).width = 256 * 20
    # 合并单元格写入
    sheet_wt_staff.write_merge(r1=1, r2=8, c1=1, c2=1, label="收\n入", style=merge_style)
    sheet_wt_staff.write_merge(r1=9, r2=14, c1=1, c2=1, label="扣\n款", style=merge_style)
    sheet_wt_staff.write_merge(r1=15, r2=15, c1=1, c2=2, label="实发", style=field_style)

workbook_wt.save(xls_filename_wt) # 保存 Excel

print("文件 %s 生成成功" % xls_filename_wt)

# Python总结

**<font color=blue size=4> 条件判断语法 if 的用法</font>**

<font color=blue size=4> 1、两个分支</font>
<pre>
if 条件 ?:
   run(A)
else:
   run(B)
</pre>
<hr/>

<font color=blue size=4> 2、三个分支（多分支）</font>
<pre>
if 条件1 ?:
   run(A)
elif 条件2 ?:
   run(B)
else:
   run(B)
</pre>

# 完整代码

In [ ]:
# 导入Excel读写工具包
import xlrd
import xlwt

# 从（from）本地 xlwt_styles.py（.py扩展名省略）文件中，导入（import）创建样式的函数（create_style）
from xlwt_styles import create_style

# 定义两个变量（xls_filename_rd、xls_filename_wt）
xls_filename_rd = "./files/Excel_工资单明细.xls" # 读取的 Excel 文件名
xls_filename_wt = "./files/Excel_员工工资单_02.xls" # 新建的 Excel 文件名
# *******************************
# 虽然变量命名只要符合命名规则即可，但建议变量命名最好要有意义，对读懂代码很有帮助
# xls:Excel扩展名  filename:文件名  rd:read读取  wt:write写入
# *******************************

# 读取 Excel（xls_filename_rd 变量刚才已经定义），得到工作簿，赋值给变量 workbook_rd
workbook_rd = xlrd.open_workbook(xls_filename_rd)

# 获取第一个工作表，赋值给变量 sheet_rd
sheet_rd = workbook_rd.sheet_by_index(0)

# 获取工作表中表格的所有字段（第一行是字段名），记住后面会用到
field_names = sheet_rd.row_values(0)
print("字段名：",field_names)  # 打印一下看看字段名

# 创建两个“空”列表变量
staff_names = [] # 员工姓名列表
staff_values_list = [] # 员工数据明细列表（这个就是列表中的列表）

# 因为第一行（索引=0）是字段名，所以从第二行（索引=1）开始读数据
for row_index in range(1, sheet_rd.nrows):
    values = sheet_rd.row_values(row_index)  # 获取员工的数据列表
    # print(values)           # 可以删除注释，打印这一行数据列表看看
    staff_name = values[2]    # 第三列是姓名
    # print(staff_name)       # 可以删除注释，打印一下看看
    staff_names.append(staff_name)  # append
    staff_values_list.append(values) # 列表中可以再包含列表 [[...],[...]]

# 打印读取的员工姓名列表、员工数据明细列表
# print("员工姓名列表：",staff_names)           # 可以删除注释，打印一下看看
# print("员工数据明细列表：",staff_values_list) # 可以删除注释，打印一下看看

# 创建写入工作簿对象
workbook_wt = xlwt.Workbook(encoding='utf-8')

# 看看要生成的表格样式，共有4个
# 合并单元格的样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=居中）
merge_style = create_style(back_color=57, font_color=1, font_size=12, horz=2)  # 标题显示用黄色背景图案
# 字段样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=靠右）
field_style = create_style(back_color=57, font_color=1, font_size=12, horz=3)
# 数据值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=居中）
value_style = create_style(back_color=1, font_color=0, font_size=12, horz=2)
# 金额值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=靠右，人民币格式）
rmb_style = create_style(back_color=1, font_color=0, font_size=12, horz=3, num_format_str="￥#,##0.00;￥-#,##0.00")

# 第一循环：遍历每个员工的姓名，每个姓名创建一个工作表
for staff_index, staff_name in enumerate(staff_names):
    # 创建工作表，工作表名就是员工姓名
    sheet_wt_staff = workbook_wt.add_sheet(staff_name)

    # 获取员工的数据明细
    staff_data = staff_values_list[staff_index]
    # 第二循环：遍历字段和数据
    for field_index, field_name in enumerate(field_names):
        row = field_index + 1 # field_index 从 0 开始，则 row 从 1 开始

        if field_index < 14: # “实发”是最后一个字段名（索引=14），忽略不写入，后面合并单元格再写入
            # 第3列（c=2）写入字段名
            sheet_wt_staff.write(r=row, c=2, label=field_name, style=field_style)
        else:
            pass # 通过，这个代码什么都不干，对于“实发”字段名就不写入了

        value = staff_data[field_index]

        if field_index <= 2:   # 0，1，2 三个字段分别是月份、部门、姓名
            sheet_wt_staff.write(r=row, c=3, label=value, style=value_style) # 第4列（c=3）写入数据
        else:    # 3、4、5...都是金额字段，用RMB货币样式            
            sheet_wt_staff.write(r=row, c=3, label=value, style=rmb_style) # 第4列（c=3）写入数据

    # 设置字段列和数据列的列宽 256 × 宽度默认单位（字符）
    sheet_wt_staff.col(1).width = 256 * 12
    sheet_wt_staff.col(2).width = 256 * 15
    sheet_wt_staff.col(3).width = 256 * 20
    # 合并单元格写入
    sheet_wt_staff.write_merge(r1=1, r2=8, c1=1, c2=1, label="收\n入", style=merge_style)
    sheet_wt_staff.write_merge(r1=9, r2=14, c1=1, c2=1, label="扣\n款", style=merge_style)
    sheet_wt_staff.write_merge(r1=15, r2=15, c1=1, c2=2, label="实发", style=field_style)

workbook_wt.save(xls_filename_wt) # 保存 Excel

print("文件 %s 生成成功" % xls_filename_wt)


# 思考题


**<font color=blue size=4>上面的课程是生成一个工作簿(Workbook)，多个工作表（工作表名称为员工姓名）<br/>
    现在的要求是：一个员工生成一个工作簿(Workbook)，工作簿命名包含员工姓名，如下图：Excel_陈浩的工资单.xls"</font>**

<font color=red size=4>这个思考题对于初学者可能会有点困难，代码虽然改起来非常简单，就是看怎么改，希望大家尝试一下改上面的代码</font>




**<font color=blue>新建Excel的内容（合并单元格）</font>**<br/>
**<font color=red>为每个员工创建一个工作簿文件，命名包含员工姓名</font>**<br/>
<img src="images/Excel工资单_03.png" align="left"></img>


# 请在下面给出思考题的代码

In [ ]:
# 导入Excel读写工具包
import xlrd
import xlwt

# 从（from）本地 xlwt_styles.py（.py扩展名省略）文件中，导入（import）创建样式的函数（create_style）
from xlwt_styles import create_style

# 读取 Excel（xls_filename_rd 变量刚才已经定义），得到工作簿，赋值给变量 workbook_rd
workbook_rd = xlrd.open_workbook(xls_filename_rd)

# 获取第一个工作表，赋值给变量 sheet_rd
sheet_rd = workbook_rd.sheet_by_index(0)

# 获取工作表中表格的所有字段（第一行是字段名），记住后面会用到
field_names = sheet_rd.row_values(0)
print("字段名：",field_names)  # 打印一下看看字段名

# TODO:请在这里自己实现，可以从上面学过的代码中复制粘贴，不要从下面思考题答案代码复制粘贴
.....

# 思考题正确答案

In [ ]:
# 导入Excel读写工具包
import xlrd
import xlwt

# 从（from）本地 xlwt_styles.py（.py扩展名省略）文件中，导入（import）创建样式的函数（create_style）
from xlwt_styles import create_style

xls_filename_rd = "./files/Excel_工资单明细.xls" # 读取的 Excel 文件名


# 读取 Excel（xls_filename_rd 变量刚才已经定义），得到工作簿，赋值给变量 workbook_rd
workbook_rd = xlrd.open_workbook(xls_filename_rd)

# 获取第一个工作表，赋值给变量 sheet_rd
sheet_rd = workbook_rd.sheet_by_index(0)

# 获取工作表中表格的所有字段（第一行是字段名），记住后面会用到
field_names = sheet_rd.row_values(0)
# print("字段名：",field_names)  # 打印一下看看字段名

# 创建两个“空”列表变量
staff_names = [] # 员工姓名列表
staff_values_list = [] # 员工数据明细列表（这个就是列表中的列表）

# 因为第一行（索引=0）是字段名，所以从第二行（索引=1）开始读数据
for row_index in range(1, sheet_rd.nrows):
    values = sheet_rd.row_values(row_index)  # 获取员工的数据列表
    staff_name = values[2]    # 第三列是姓名
    staff_names.append(staff_name)  # append
    staff_values_list.append(values) # 列表中可以再包含列表 [[...],[...]]

# *******************************************************************
# 注意：以下代码不用了，注释掉了，知道原因吗？
# workbook_wt = xlwt.Workbook(encoding='utf-8')

# 看看要生成的表格样式，共有4个
# 合并单元格的样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=居中）
merge_style = create_style(back_color=57, font_color=1, font_size=12, horz=2)  # 标题显示用黄色背景图案
# 字段样式（背景=草绿，文字颜色=白色，字号=12，水平对齐=靠右）
field_style = create_style(back_color=57, font_color=1, font_size=12, horz=3)
# 数据值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=居中）
value_style = create_style(back_color=1, font_color=0, font_size=12, horz=2)
# 金额值的样式（背景=白色，文字颜色=黑色，字号=12，水平对齐=靠右，人民币格式）
rmb_style = create_style(back_color=1, font_color=0, font_size=12, horz=3, num_format_str="￥#,##0.00;￥-#,##0.00")

# 第一循环：遍历每个员工的姓名，每个员工创建一个工作簿文件
for staff_index, staff_name in enumerate(staff_names):
    # *******************************************************************
    # 创建写入工作簿对象（放到循环内了，因为一个循环需要新建一个Excel文件）
    # *******************************************************************
    workbook_wt = xlwt.Workbook(encoding='utf-8')
    
    # 创建工作表，工作表名就是员工姓名
    sheet_wt_staff = workbook_wt.add_sheet(staff_name)

    # 获取员工的数据明细
    staff_data = staff_values_list[staff_index]
    # 第二循环：遍历字段和数据
    for field_index, field_name in enumerate(field_names):
        row = field_index + 1 # field_index 从 0 开始，则 row 从 1 开始

        if field_index < 14: # “实发”是最后一个字段名（索引=14），忽略不写入，后面合并单元格再写入
            # 第3列（c=2）写入字段名
            sheet_wt_staff.write(r=row, c=2, label=field_name, style=field_style)
        else:
            pass # 通过，这个代码什么都不干，对于“实发”字段名就不写入了

        value = staff_data[field_index]

        if field_index <= 2:   # 0，1，2 三个字段分别是月份、部门、姓名
            sheet_wt_staff.write(r=row, c=3, label=value, style=value_style) # 第4列（c=3）写入数据
        else:    # 3、4、5...都是金额字段，用RMB货币样式            
            sheet_wt_staff.write(r=row, c=3, label=value, style=rmb_style) # 第4列（c=3）写入数据

    # 设置字段列和数据列的列宽 256 × 宽度默认单位（字符）
    sheet_wt_staff.col(1).width = 256 * 12
    sheet_wt_staff.col(2).width = 256 * 15
    sheet_wt_staff.col(3).width = 256 * 20
    # 合并单元格写入
    sheet_wt_staff.write_merge(r1=1, r2=8, c1=1, c2=1, label="收\n入", style=merge_style)
    sheet_wt_staff.write_merge(r1=9, r2=14, c1=1, c2=1, label="扣\n款", style=merge_style)
    sheet_wt_staff.write_merge(r1=15, r2=15, c1=1, c2=2, label="实发", style=field_style)
    # *******************************************************************
    # 一个循环需要新建一个Excel文件，文件名变化的部分就是员工姓名
    xls_filename_wt = "./files/Excel_%s的工资单.xls" % staff_name    
    workbook_wt.save(xls_filename_wt) # 保存 Excel
    print("文件 %s 生成成功" % xls_filename_wt)
    # *******************************************************************